<a href="https://colab.research.google.com/github/Luizcarlosqueiroz/Otimizacao_por_Enxame_de_Particulas/blob/main/Otimiza%C3%A7%C3%A3o_por_Enxame_de_Part%C3%ADculas_Rastrigin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importar as Bibliotecas Básicas

In [1]:
import random
import math
import matplotlib.pyplot as plt
import numpy as np 

## Função Objetivo (Rastrigin)
$$f(x_1 \cdots x_n) = 10n + \sum_{i=1}^n (x_i^2 -10cos(2\pi x_i))$$
$$-5.12 \leq x_i \leq 5.12$$ 

In [2]:
def funcao_objetivo(X):
  A = 10
  n = 2
  y = A*n + sum([(x**n - A * np.cos(n * math.pi * x)) for x in X])
  return y

In [3]:
a = np.array([1,2,3,4])
funcao_objetivo(a)

10.0

### Posição Inicial e Atualização

In [4]:
def initPosition(Np, Nd, xMin, xMax):
  return xMin + np.random.rand(Np, Nd)*(xMax-xMin)


In [5]:
initPosition(10,3)

TypeError: ignored

In [11]:
def updatePosition (R, V, Np, Nd, xMin, xMax):

  R += V

  # Caso a particula fuja do domínio de busca desejado
  for particle in range(Np):
    for dimension in range(Nd):
      if R[particle][dimension] > xMax:
        R[particle][dimension] = xMax
      if R[particle][dimension] < xMin:
        R[particle][dimension] = xMin
        
  return R

### Velocidade e Atualização

In [6]:
def initVelocity(Np, Nd, vMin, vMax):
    V = vMin + np.random.rand(Np, Nd)*(vMax-vMin)
    return V

In [7]:
def updateVelocity(R, V, Np, Nd, w, c1, c2, vMin, vMax, chi, pBestPos, gBestPos):
    
    r1 = np.random.rand()
    r2 = np.random.rand()

    for particle in range(Np):
        V[particle, :] = chi * (w * V[particle, :] + r1*c1*(pBestPos[particle, :] - R[particle, :]) + r2*c2*(gBestPos - R[particle, :]))

        for dimension in range(Nd):
            if V[particle][dimension] > vMax: 
                V[particle][dimension] = vMax
            if V[particle][dimension] < vMin: 
                V[particle][dimension] = vMin

    return V

In [8]:
def updateFitness(R, Np, gBestValue, gBestPos, pBestValue, pBestPos):

    for particle in range(Np):
        M = funcao_objetivo(R[particle, :])

        if M < gBestValue:
            gBestValue = M
            gBestPos = R[particle, :]

        if M < pBestValue[particle]:
            pBestValue[particle] = M
            pBestPos[particle, :] = R[particle, :]

    return gBestValue, gBestPos, pBestValue, pBestPos

## Código Final

In [12]:
lista_de_gbests = []

for _ in range(30):

  #Número de particulas (mudando do padrão para teste), Número de dimensões, Número de Iterações
  Np, Nd, Nt = 25, 2, 500
  #Pesos para atualizar a velocidade (seguindo o padrão)
  c1, c2 = 2.05 , 2.05
  #Definição do min e max do coeficiente de inércia
  wMin, wMax = 0.4, 0.9

  #Espaço (de acordo com os limites dados pela função de Rastring)
  xMin, xMax = -5.12 , 5.12
  #Velocidades (imaginando que as particulas podem se mover menos, dado um espaço amostral menor)
  vMin, vMax = 0.15*xMin , 0.15*xMax

  #Visto que o problema é de minimização, inicializando os best's com valor muito altos
  gBestValue = float("inf")
  pBestValue = [float("inf")] * Np

  pBestPos = np.zeros((Np, Nd))
  gBestPos = np.zeros(Nd)

  #Calculando o fator de Clerck
  #Considerando phi > 4, o chi será de 0.7, fazendo com que a V esteja sempre diminuindo
  phi = c1 + c2
  chi = 2.0/abs(2.0-phi-np.sqrt(pow(phi, 2)-4*phi))

  #Inicializando as particulas e velocidade
  R = initPosition(Np, Nd, xMin, xMax)
  V = initVelocity(Np, Nd, vMin, vMax)

  history = []

  for j in range (0, Nt):

    R = updatePosition(R, V, Np, Nd, xMin, xMax)

    gBestValue, gBestPos, pBestValue, pBestPos = updateFitness(R, Np, gBestValue, gBestPos, pBestValue, pBestPos)
    history.append(gBestValue)

    w = wMax - ((wMax-wMin)/Nt)*j
    V = updateVelocity(R, V, Np, Nd, w, c1, c2, vMin, vMax, chi, pBestPos, gBestPos)

  lista_de_gbests.append(gBestValue)

In [13]:
lista_de_gbests

[0.0,
 0.0,
 0.9949590570932898,
 0.0,
 0.0,
 2.5828228444879642e-11,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9949590570932898,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9949590570932898,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9949590570932898,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

### Conclusão interessante: O resultado de 100 e 500 iterações são difere muito. Mas com poucas iterações trazem resultados 'ruins'

### Tentativas frustradas de rodar a API

In [ ]:
!pip install pyswarms
import pyswarms as ps
from pyswarms.utils.functions import single_obj as fx

In [ ]:
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9, 'k': 2, 'p': 2}

In [ ]:
optimizer = ps.single.GlobalBestPSO(n_particles=10, dimensions=2, options=options , bounds=bounds)

In [ ]:
cost, pos = optimizer.optimize(fx.sphere, iters=1000)

2021-11-15 21:28:29,035 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9, 'k': 2, 'p': 2}
pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=3.39e-79
2021-11-15 21:28:30,891 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 3.386393126192514e-79, best pos: [ 4.96416102e-40 -3.03661599e-40]


In [ ]:
max_bound = 5.12 * np.ones(2)
min_bound = - max_bound
bounds = (min_bound, max_bound)
#numero de variavies
nv = 2
# considerando que é uma minimização, negativo
mm = -1 